## GEOSeMOSYS - exploring linear programming geospatial energy modelling 

Nandi Moksnes, Mark Howells, Alexandros Korkovelos, Will Usher

This Jupyter Notebook is dectription of how to reproduce the results in the paper:
For any questions please contact Nandi Moksnes: nandi@kth.se

## 1. The files in "GIS_data" are downloaded and placed in a "temp" folder.

In [ ]:
%load_ext autoreload
%autoreload
from Download_files import *
import os
current = os.getcwd()
os.chdir(current)
print(os.getcwd())

download_url_data('input_data/GIS_URL.txt', 'temp')
unzip_all('input_data/GIS_URL.txt')

## 2. The files are then projected and clipped to the administrative boundaries. 
Approx 6 hours

In [ ]:
%autoreload
import os
from Project_GIS import *
current = os.getcwd()
os.chdir(current)

main('../GIS_data')

## 3. Through QGIS make raster to point layer and save

1) masked_UMT37S_ken_ppp_2018_1km_Aggregated: SAGA raster to point 

This step is downloaded from zenodo so it is not needed for the user to execute.

Run time: Approx 1 minute


In [ ]:
%autoreload
from Download_files import *
import os
current = os.getcwd()
os.chdir(current)
print(os.getcwd())
download_url_data('input_data/zenodo.txt', '/Projected_files')

## 4. The GIS layers are prepared to for a heuristic approximation for electrified settlements
Approx 15 min

In [ ]:
%autoreload
import matplotlib.pyplot as plt
from settlement_build import *
current = os.getcwd()
os.chdir(current)

pop_shp = '../Projected_files/raster_to_point_UMT37S_Kenya.shp'
Projected_files_path = '../Projected_files'

rasterize = raster_proximity(Projected_files_path)
points = raster_to_point(rasterize, pop_shp, Projected_files_path)

## 5. Approximate location of urban settlements and the electrified settlements 1kmx1km resolution
Approx 10 min. Make sure to wait so that the shapefile and csv file is saved properly.

In [ ]:
%load_ext autoreload
%autoreload
from elec_start import *
import os
current = os.getcwd()
os.chdir(current)

elec_actual = 0.75  # percent
pop_cutoff = 4000  # people
dist_minigrid = 5000  # meters
dist_mv = 5000 #meters
dist_lv = 2000 #meters
dist_to_trans = 5000  # meters
dist_to_sub = 5000 #meters
dist_minig = 5000 #meters
min_night_lights = 20
max_grid_dist = 10000  # meters
max_road_dist = 300  # meters
pop_cutoff2 = 36000 # people
urban_elec_ratio = 83.5  # percent
rural_elec_ratio = 71.5  # percent
pop_actual = 52570000  # peolpe
urban = 0.275  # percent
urban_cutoff = 20000
start_year = 2018
settlement = gpd.read_file("../Projected_files/settlements.shp")

settlements = pd.DataFrame(settlement, copy=True)
urbansettlements = calibrate_pop_and_urban(settlements, pop_actual, urban, urban_cutoff)
elec_current_and_future(urbansettlements, elec_actual, pop_cutoff, dist_to_trans, dist_to_sub, dist_minig, min_night_lights,
                            max_grid_dist, urban_elec_ratio, rural_elec_ratio, max_road_dist, pop_actual, pop_cutoff2, start_year, dist_mv, dist_lv)


In [ ]:
%autoreload
import geopandas as gpd
import matplotlib.pyplot as plt

shp_path = (r'..\Projected_files\elec.shp')

point = gpd.read_file(shp_path)

point.head()

In [ ]:
fig, ax = plt.subplots(1, 1)
point.plot(column='elec', ax=ax)
fig.suptitle('Estimated electrified popluation (in yellow) Kenya', fontsize=18)

## 6. Download Renewable.ninja data for all 378 locations

In [ ]:
%autoreload
from renewable_ninja_download import *

shapefile = '../Projected_files/new_40x40points_WGSUMT37S.shp'
path = "temp"
coordinates = project_vector(shapefile)
wind, solar = csv_make(coordinates)
down = download(path, wind, solar)


## 7. Scale down from1x1 km to 40 x 40 km electrified and unelectrified demand cells

In [ ]:
%autoreload
from Demand import *
shape = '../Projected_files/new_40x40polygon_WGSUMT37S.shp'
gdp = '../Projected_files/masked_UMT37S_GDP_PPP_Layer2015.tif'
elec_shp = '../Projected_files/elec.shp'

demand_cells = join(elec_shp, gdp, shape)

# 8. Calculate the demand per cell based on following equations

![title](input_data/eq1.png)
![title](input_data/eq2.jpg)

The reference scenario follows the reference scenario from [Updated least cost power development plan 2017-2037](http://gak.co.ke/wp-content/uploads/2019/02/Updated-Least-Cost-Power-Development-Plan-2017-2022-min.pdf) but deducts the un-electrified scenario which is multiplied with the [Tier 2](http://gak.co.ke/wp-content/uploads/2019/02/Updated-Least-Cost-Power-Development-Plan-2017-2022-min.pdf ) and [4 people per household](https://openknowledge.worldbank.org/bitstream/handle/10986/24368/Beyond0connect0d000technical0report.pdf?sequence=1&isAllowed=y). Full electrification is achived by 2030 

The vision scenario follows the vision scenario from [Updated least cost power development plan 2017-2037](http://gak.co.ke/wp-content/uploads/2019/02/Updated-Least-Cost-Power-Development-Plan-2017-2022-min.pdf) but deducts the un-electrified scenario which is multiplied with the [Tier 3](http://gak.co.ke/wp-content/uploads/2019/02/Updated-Least-Cost-Power-Development-Plan-2017-2022-min.pdf ) and [4 people per household](https://openknowledge.worldbank.org/bitstream/handle/10986/24368/Beyond0connect0d000technical0report.pdf?sequence=1&isAllowed=y). Full electrification is achieved 2022 

# 9. Transmission lines

From the demand_cell.csv file identify the cells that does not have any electrified cells in the 40x40 cell and classify them as unelectrified. The cells that has one electrified cell is concidered to have an HV line or possibility to connect to an additional LV line and connect the remainder of the area.

Cost concidered are [2500 USD MW-km and 1500 USD/MW](https://iea-etsap.org/E-TechDS/PDF/E12_el-t&d_KV_Apr2014_GSOK.pdf) and  [4000 USD/MW-km](https://iea-etsap.org/E-TechDS/PDF/E12_el-t&d_KV_Apr2014_GSOK.pdf) for low voltage lines.

The cells that are electrified and not 5000 m from a minigrid the cell is concidered electrified by transmission lines.
The results is that from 378 cells to 202 un-electrified cells (16 are concidered connected to minigrid).

In [ ]:
%autoreload
from Transmissionlines import * 

pop_shp = 'new_40x40points_WGSUMT37S.shp'
transmission_near = "run/Demand/transmission.shp"
un_elec_cells = 'run/un_elec_cells.csv'
Projected_files_path = '../Projected_files/'

near_dist(pop_shp, un_elec_cells, Projected_files_path)